In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model

import tensorflow as tf

import pandas as pd

import os
import pickle
import numpy as np

import scipy.sparse as sp
import scipy.io as spio

import isolearn.io as isoio
import isolearn.keras as iso

from aparent.data.aparent_data_array import load_data


Using TensorFlow backend.


In [3]:
#Load designed MPRA data

data_version = '_seq'

file_path = '../../data/prepared_data/apa_array_data/'
array_gens = load_data(batch_size=1, file_path=file_path, data_version=data_version)


Designed MPRA size = 186066
Training set size = 0
Validation set size = 0
Test set size = 186066


In [4]:
#Load APARENT model

#model_name = 'aparent_large_lessdropout_all_libs_no_sampleweights'
#model_name = 'aparent_large_all_libs'
model_name = 'aparent_libs_30_31_34'

save_dir = os.path.join(os.getcwd(), '../../aparent/saved_models')
model_path = os.path.join(save_dir, model_name + '.h5')

aparent_model = load_model(model_path)

/home/jlinder2/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
#Predict from test data generator

iso_pred_test, cut_pred_test = aparent_model.predict_generator(array_gens['all'], workers=4, use_multiprocessing=True)

#Calculate isoform logits
logodds_pred_test = np.ravel(np.log(iso_pred_test / (1. - iso_pred_test)))


In [8]:
#Copy the test set dataframe and store isoform predictions

array_df = array_gens['all'].sources['df'].reset_index().copy()

array_df['iso_pred'] = iso_pred_test
array_df['logodds_pred'] = logodds_pred_test

array_df = array_df[['seq', 'master_seq', 'iso_pred', 'logodds_pred']]


In [12]:
#Dump prediction dataframe and cut probability matrix

isoio.dump({'array_df' : array_df, 'cut_prob' : sp.csr_matrix(cut_pred_test)}, 'apa_array_data/' + model_name + '_predictions' + data_version)